---
<div align="center">

# Connect-4 [Artificial Intelligence Project]
</div>

---

<div align="center">

## Search Problem 
</div>

A Search Problem is composed by multiple factors:
- States (Any possible configuration of the Board)
- Initial and Goal States
- Action Space (actions and their effects on the environment)
- Action Cost

---

In [1]:
import numpy as np
from copy import (deepcopy)
import heapq
import random as rd
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Connect_Four_State:
    # Defining Board's Size
    NROWS = 6
    NCOLS = 7
    
    def __init__(self):
        # Matrix to Store the Board's Values
        self.board = np.zeros(shape=(self.NROWS, self.NCOLS), dtype=np.int8)

        # Array to keep track of the row's index for every column where a piece can be placed
        # Basically it's the Row idx for every column's height
        self.columns_height = np.full(shape=self.NCOLS, fill_value=(self.NROWS - 1), dtype=np.int8)

        # Defining the Possible Actions (Initially a piece can be placed in any column)
        self.actions = [col for col in range(self.NCOLS)]
        
        # Initializing a variable to track the current player
        self.current_player = 1
        
        # Variable to store the Winner (-1 - Game still running || 0 - Tie || 1 - PLayer 1 || 2 - Player 2 / AI)
        self.winner = -1
    
    def next_player(self):
        # Returns the next turn's player
        return 3 - self.current_player

    def is_over(self):
        # If the Winner corresponds to -1 then the game is not finished otherwise it is
        return self.winner != -1

    def reset(self):
        # Calls back the Constructor
        return self.__init__()
    
    def inside_board(self, x, y):
        # Checks if a position (x,y) exists inside the board's matrix
        return (x >= 0 and x < self.NROWS) and (y >= 0 and y < self.NCOLS)
    
    def move(self, ncol):
        # Creating a new state
        new_state = deepcopy(self)

        # Checks of the given column can be played on
        if(ncol in self.actions):
            # Inserting the move into the board
            new_state.board[new_state.columns_height[ncol]][ncol] = new_state.current_player
            
            # Updating the "ncol"'s height
            new_state.columns_height[ncol] -= 1
            
            # Checking if the column is full and therefore uncapable of receiving more pieces -> Changes the action space
            if (new_state.columns_height[ncol] < 0):
                new_state.actions.remove(ncol)

            # Updates the current Winner
            new_state.update_winner()

            # Updating current player for the next state
            new_state.current_player = new_state.next_player()
            
        # Returns the New State (Could be the same as the previous if the action could not be performed)
        return new_state

    def generate_new_states(self):
        # List to contain all the new states
        new_states = []

        # Iterates through all possible actions and creates a new state for each one
        for ncol in self.action:
            new_states.append(self.move())

        # Returns all expanded states
        return new_states
    
    def check_line(self, n, player, values):
        # -> Checks for a 4-piece line given 4 consecutive values from the matrix

        # Calculates the number of pieces from the value's array that are from the given player
        player_pieces = sum(list(map(lambda piece: piece == player, values)))
        
        # Finds a Full 4-piece Line
        if n == 4:
            return player_pieces == 4
        
        # Finds a complete 3-piece Line
        if n == 3:
            # Checks if it's possible to make it a 4-piece Line by adding another
            empty_places = sum(list(map(lambda piece: piece == 0, values)))
            return player_pieces == 3 and empty_places == 1
    
    def count_lines(self, n, player):
        # -> Searches the Board looking for a 4-piece Combination
        
        # Initializes the number of lines found
        total_lines = 0

        # Loops through the board
        for row in range(self.NROWS):
            for col in range(self.NCOLS):
                # Found a Horizontal Line
                if col < self.NCOLS - 3 and self.check_line(n, player, [self.board[row][col + i] for i in range(4)]):
                    total_lines += 1
                
                # Found a Vertical Line
                if row < self.NROWS - 3 and self.check_line(n, player, [self.board[row + i][col] for i in range(4)]):
                    total_lines += 1
                
                # Found a Descending Diagonal Line
                if row < self.NROWS - 3 and col < self.NCOLS - 3 and self.check_line(n, player, [self.board[row + i][col + i] for i in range(4)]):
                    total_lines += 1
                
                # Found a Ascending Diagonal Line
                if col < self.NCOLS - 3 and row > 3 and self.check_line(n, player, [self.board[row - i][col + i] for i in range(4)]):
                    total_lines += 1
        
        return total_lines
    
    def update_winner(self):
        # -> Updates the Current State's Winner
        
        # Checks for a 4-piece combination made by PLayer 1
        if self.count_lines(4, 1) > 0:
            self.winner = 1
        
        # Checks for a 4-piece combination made by PLayer 1
        elif self.count_lines(4, 2) > 0:
            self.winner = 2

        # Checks for Possible moves 
        if (len(self.actions) == 0):
            self.winner = 0

    """ AUXILIAR METHODS """

    def __str__(self):
        # -> Converts the board into the style used in the Assignment 1 Paper
        DECODER = {0:'-', 1:'X', 2:'O'} 
        line = ["-" for i in range(2*self.NCOLS -1)]
        line.insert(0, '#')
        line.insert(1, ' ')
        line.insert(len(line), ' ')
        line.insert(len(line), '#')
        formated_line = "".join(line)
        new_board = formated_line + '\n'
        for x in range (self.NROWS):
            for y in range (self.NCOLS):
                if (y == 0):
                    new_board += "| " + DECODER[self.board[x][y]]
                elif (y == self.NCOLS -1):
                    new_board += " " + DECODER[self.board[x][y]] + " |"
                else:
                    new_board += " " + DECODER[self.board[x][y]]
            new_board += '\n'
        new_board += formated_line
        return new_board
        # return str(self.board)

    def __hash__(self):
        return hash(str(self.board))

    def __eq__(self, other:object):
        if (not isinstance(other, Connect_Four_State)):
            return False
        return hash(self) == hash(other)

In [3]:
game = Connect_Four_State()
print(game)

# ------------- #
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
# ------------- #


In [4]:
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
print(game)

# ------------- #
| - O - - - - - |
| - X - - - - - |
| - O - - - - - |
| - X - - - - - |
| - O - - - - - |
| - X - - - - - |
# ------------- #


---
<div align="center">

## TreeNode Class
</div>

---

In [5]:
class TreeNode:
        
    def __init__(self, state, parent=None):
        # Stores a State of the Game
        self.state = state

        # Keeps a reference to his Parent Node
        self.parent = parent

        # Stores all the generated nodes
        self.children = []
    
    def generate_new_node(self, ncol):
        # Creates a new state after the move
        new_state = self.state.move(ncol)

        # Wraps it with a TreeNode 
        new_node = TreeNode(state=new_state, parent=self)
        
        # Inserts the New Node into the Current Node's Children
        self.children.append(new_node)
        
        # Returns the generated Node
        return new_node

    def __str__(self):
        return str(self.state)
    
    def __hash__(self):
        return hash(str(self.state) + str(self.parent) + "".join([child for child in self.children]))
    
    def __eq__(self, other:object):
        if (not isinstance(other, TreeNode)):
            return False
        return self.__hash__() == other.__hash__()

In [6]:
initial_state = Connect_Four_State()
new_node = TreeNode(initial_state)

---
<div align="center">

## Heuristics
</div>

---

In [7]:
""" SCORE FUNCTIONS - NOT 100% SURE [I THINK IT'S CALCULATING SOME PIECES MORE THAN ONCE INSIDE THE SAME LINE TYPE] """

def check_line_configuration(player, values):
    # Given a Array of values if counts the amount of pieces from both players and also the empty spaces
    player_pieces = sum(list(map(lambda piece: piece == player, values)))
    enemy_player_pieces = sum(list(map(lambda piece: piece == (3 - player), values)))
    empty_spaces = sum(list(map(lambda piece: piece == 0, values)))
    return (player_pieces, enemy_player_pieces, empty_spaces)

def calculate_line_score(player_pieces, enemy_pieces, empty_spaces):
    # -> Calculates the score to return based on the line configuration

    # Defining a Score Decoder for the amount of Empty Spaces
    SCORE_DECODER = {0:512,  # 0 Empty Spaces - There are 4 player's pieces
                     1:50,   # 1 Empty Space  - There are 3 player's pieces
                     2:10,   # 2 Empty Spaces - There are 2 player's pieces
                     3:1,    # 3 Empty Spaces - There is 1 player's pieces
                     4:0}    # 4 Empty Spaces - There are only empty spaces

    if (player_pieces > 0):
        # We have both player's pieces
        if (enemy_pieces > 0):
            return 0
        # There are no enemy pieces
        else:
            return SCORE_DECODER[empty_spaces]
    else:
        # Only have empty pieces
        if (enemy_pieces == 0):
            return 0
        else:
            return - SCORE_DECODER[empty_spaces]

def calculate_board_score(state:Connect_Four_State):
    # Calculates current State Evaluation
        
    # Initializes the number of lines found
    total_score = 0

    # Loops through the board
    for row in range(state.NROWS):
        for col in range(state.NCOLS):
            # Checks a Horizontal Line
            if col < state.NCOLS - 3:
                (player_pieces, enemy_player_pieces, empty_spaces) = state.check_line_configuration(state.current_player, [state.board[row][col + i] for i in range(4)])
                total_score += state.calculate_line_score(player_pieces, enemy_player_pieces, empty_spaces)
            
            # Checks a Vertical Line
            if row < state.NROWS - 3:
                (player_pieces, enemy_player_pieces, empty_spaces) = state.check_line_configuration(state.current_player, [state.board[row + i][col] for i in range(4)])
                total_score += state.calculate_line_score(player_pieces, enemy_player_pieces, empty_spaces)
            
            # Checks a Descending Diagonal Line
            if row < state.NROWS - 3 and col < state.NCOLS - 3:
                (player_pieces, enemy_player_pieces, empty_spaces) = state.check_line_configuration(state.current_player, [state.board[row + i][col + i] for i in range(4)])
                total_score += state.calculate_line_score(player_pieces, enemy_player_pieces, empty_spaces)

            # Checks a Ascending Diagonal Line
            if col < state.NCOLS - 3 and row > 3:
                (player_pieces, enemy_player_pieces, empty_spaces) = state.check_line_configuration(state.current_player, [state.board[row - i][col + i] for i in range(4)])
                total_score += state.calculate_line_score(player_pieces, enemy_player_pieces, empty_spaces)
    
    return total_score

---
<div align="center">

## Algorithms
</div>

---

### A* Search

In [8]:
def A_Star_Search(node:TreeNode):
    """ A* Search """
    pass

### MinMax - MAYBE????

In [9]:
def MinMax(node:TreeNode):
    """ MinMax with Alpha-Beta Prunning - EXTRA """
    pass

### Monte Carlo Tree Search

In [10]:
def MCTS(node:TreeNode):
    """ Monte Carlo Tree Search """
    pass

---
<div align="center">

## Connect_Four [Main Class]
</div>

---

In [11]:
class Connect_Four:

    # Screen Parameters
    SQSIZE = 80
    X_OFFSET = 60
    Y_OFFSET = 100
    BORDER_THICKNESS = 10
    
    # Circle - Screen Parameters
    CIRCLE_OFFSET = 10
    CIRCLE_POS = (X_OFFSET + SQSIZE//2, Y_OFFSET + SQSIZE//2)
    CIRCLE_RADIUS = (SQSIZE//2) - CIRCLE_OFFSET

    # RGB Colors
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)
    
    LIGHT_BLUE = (135, 206, 250)
    BLUE = (102, 178, 255)
    DARK_BLUE = (0, 76, 153)
    
    RED = (189, 22, 44)
    DARK_RED = (151, 18, 35)
    
    GREEN = (0, 204, 102)
    DARK_GREEN = (0, 153, 76)

    # Defining a Array with the Piece's Colors [Tuple: (BORDER, INNER CIRCLE)]
    PIECES_COLORS = [(DARK_BLUE, BLUE),        # Empty Pieces
                     (DARK_RED, RED),          # Player 1
                     (DARK_GREEN, GREEN)]      # Player 2
    
    def __init__(self, player1="Player", player2="Player"):
        self.root = TreeNode(state=Connect_Four_State())
        self.current_node = deepcopy(self.root)
        
        self.WIDTH = self.root.state.NCOLS*self.SQSIZE + 2*self.X_OFFSET
        self.HEIGHT = self.root.state.NROWS*self.SQSIZE + 2*self.Y_OFFSET

        self.player1 = player1
        self.player2 = player2
        self.PLAYER_ACTION = {1:self.player1, 2:self.player2}

        self.clicked = False

    def get_current_player_action(self):
        # Returns a string with the type of approach to be taken
        return self.PLAYER_ACTION[self.current_node.state.current_player]
    
    """ TERMINAL METHODS """
    
    def run_terminal(self):
        # Creating the Game Loop inside the Terminal (It Runs while we haven't reached a Final State)
        while not self.current_node.state.is_over():

            # Printing current board configuration
            print(f"\nCURRENT BOARD: \n{self.current_node.state}")

            # Getting the Current Action to Perform (if it's a algorithm or a player)
            current_action = self.get_current_player_action()
            
            if (current_action == "Player"):
                # Requesting a column to play at
                ncol = int(input(f"| Player {self.current_node.state.current_player} | Choose a Column to Play: "))
            
            elif(current_action == "Random"):
                # Randomizing a col to play at & printing wich one it was
                ncol = rd.randrange(0, len(self.current_node.state.actions))
                print(f"| Randomizer AI | Choosed to Play in the {ncol}th column ")

            elif(current_action == "A*_Search"):
                """ TO IMPLEMENT """ 
                pass
            
            elif(current_action == "MCTS"):
                """ TO IMPLEMENT """ 
                pass
            
            # Creating a new Node by making a move into the "ncol" column
            new_node = self.current_node.generate_new_node(ncol)

            # Updating Current Node
            self.current_node = new_node

        # Printing Final Board Configuration
        print(f"\nFINAL BOARD: \n{self.current_node.state}")
        
        # Checking if it was a Tie
        if self.current_node.state.winner == 0:
            print("Tie")
        else: # A Player Won
            print(f"-> Player {self.current_node.state.winner} Wins!")

    """ GUI METHODS """
    
    def write(self, font, text, size, color, bg_color, bold, pos, screen):
        # Writes Text into the Screen
        letra = pygame.font.SysFont(font, size, bold)
        frase = letra.render(text, 1, color, bg_color)
        screen.blit(frase, pos)
    
    def write_winner(self, screen):
        winner_name = " " + self.PLAYER_ACTION[self.current_node.state.winner] + " " + str(self.current_node.state.winner) + " Wins!"
        font_size = 50
        winner_text = len(winner_name) * font_size
        (x, y) = ((self.WIDTH - winner_text) // 2, (self.Y_OFFSET - font_size) // 2)
        self.write(font='Arial', text=winner_name, size=font_size, color=self.LIGHT_BLUE, bg_color=self.WHITE, bold=True, pos=(3*x, y), screen=screen)
    
    def draw_board(self, screen):
        # Draws Board's Shadow
        board_rect_shadow = pygame.Rect((self.X_OFFSET - self.BORDER_THICKNESS, self.Y_OFFSET - self.BORDER_THICKNESS),
                                        (self.SQSIZE*self.root.state.NCOLS + 2*self.BORDER_THICKNESS, self.SQSIZE*self.root.state.NROWS + 2*self.BORDER_THICKNESS))
        pygame.draw.rect(screen, self.DARK_BLUE, board_rect_shadow)

        # Draws Main Board
        board_rect = pygame.Rect((self.X_OFFSET, self.Y_OFFSET), (self.SQSIZE*self.root.state.NCOLS, self.SQSIZE*self.root.state.NROWS))
        pygame.draw.rect(screen, self.BLUE, board_rect)

        # Drawing Circles in the Board
        for row in range(self.current_node.state.NROWS):
            for col in range(self.current_node.state.NCOLS):
                # Getting the Colors from the Auxiliar List
                (Border_Color, Circle_Color) = self.PIECES_COLORS[self.current_node.state.board[row][col]]

                # Drawing the Board's border around the pieces
                pygame.draw.circle(screen, self.DARK_BLUE, (self.X_OFFSET + self.SQSIZE//2 + col*(self.SQSIZE),
                                                            self.Y_OFFSET + self.SQSIZE//2 + row*(self.SQSIZE)), int(1.15*self.CIRCLE_RADIUS))
                
                # Drawing the Circle's Border
                pygame.draw.circle(screen, Border_Color, (self.X_OFFSET + self.SQSIZE//2 + col*(self.SQSIZE),
                                                          self.Y_OFFSET + self.SQSIZE//2 + row*(self.SQSIZE)), self.CIRCLE_RADIUS)

                # Drawing the Main Circle
                pygame.draw.circle(screen, Circle_Color, (self.X_OFFSET + self.SQSIZE//2 + col*(self.SQSIZE),
                                                          self.Y_OFFSET + self.SQSIZE//2 + row*(self.SQSIZE)), int(0.9*self.CIRCLE_RADIUS))
    
    def draw(self, screen):
        # Filling the Background with Blue
        screen.fill(self.LIGHT_BLUE)

        # Drawing the Current Board Elements
        self.draw_board(screen)
    
    def run_gui(self, screen):
        # Create a Flag to keep track of current state of the Application / GUI
        game_run = True

        # Main Loop
        while game_run:
            
            # Draws the Game Elements into the Screen
            self.draw(screen)

            # If we haven't reached a Final State then keep playing
            if not self.current_node.state.is_over():
                
                # Getting current player action
                current_action = self.get_current_player_action()

                # Checking if the Player is Player
                if (current_action == "Player"):
                    
                    # Checking if we pressed the mouse 1 button and therefore changed the self.clicked flag
                    if not self.clicked and pygame.mouse.get_pressed()[0] == 1:
                        
                        # Getting Mouse Position
                        (y, x) = pygame.mouse.get_pos()
    
                        # Modifying the Mouse Coordinates to match with the Screen Stylling
                        x = (x - self.Y_OFFSET) // self.SQSIZE 
                        y = (y - self.X_OFFSET) // self.SQSIZE
    
                        # Checking if the coordenates exist in the board. If so, add a piece to the column that the mouse was pressed on
                        if (self.current_node.state.inside_board(x, y)):
                            self.current_node = self.current_node.generate_new_node(y)
    
                        # Updating the "clicked" flag
                        self.clicked = True
        
                    # Checking if we released the mouse 1 button and therefore changed the self.clicked flag
                    if self.clicked and pygame.mouse.get_pressed()[0] == 0:
                        
                        # Updating the "clicked" flag
                        self.clicked = False

                # Checking if the Player is set to Random
                elif(current_action == "Random"):
                    # Randomizing a col to play at
                    ncol = rd.randrange(0, len(self.current_node.state.actions))

                    # Executing the Move
                    self.current_node = self.current_node.generate_new_node(ncol)

                elif(current_action == "A*_Search"):
                    """ TO IMPLEMENT """ 
                    pass
                
                elif(current_action == "MCTS"):
                    """ TO IMPLEMENT """ 
                    pass
            
                # Delays Time before showing next move
                # pygame.time.delay(300)
            else:
                self.write_winner(screen)
            
            # Main Event Loop
            for event in pygame.event.get():
                if (event.type == pygame.QUIT):
                    
                    # pygame.quit()
                    # exit()
                    quit()
                    game_run = False
                    
            
            # Updates the Window
            pygame.display.update()

---
<div align="center">

## Graphical User Interface
</div>

---

In [12]:
class APP:

    Connect_Four_Static = Connect_Four()
    WIDTH = Connect_Four_Static.WIDTH
    HEIGHT = Connect_Four_Static.HEIGHT
    
    def __init__(self):
        self.menu = "PVR"

    def run(self):
        # Initializing Window / Screen
        pygame.init()
        screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
        pygame.display.set_caption("Connect-4")
        ICON_IMG = pygame.image.load('./Assets/Connect-Four.png').convert_alpha()
        pygame.display.set_icon(ICON_IMG)

        # Create a Flag to keep track of current state of the Application / GUI
        run = True

        # Main Loop
        while run:

            if (self.menu == "Main"):
                screen.fill(self.Connect_Four_Static.LIGHT_BLUE)
            
            if (self.menu == "PVR"):
                New_Game = Connect_Four("Player", "Random")
                New_Game.run_gui(screen)
                self.menu = "Main"
            
            # Main Event Loop
            for event in pygame.event.get():
                if (event.type == pygame.QUIT):
                    run = False
            
            # Updates the Window
            pygame.display.update()
        pygame.quit()

In [13]:
App = APP()

In [14]:
App.run()